In [1]:
%pip install llama_index.core flask python-dotenv llama-index-readers-file llama-index-embeddings-openai llama-index-llms-openai llama-index-vector-stores-milvus pymilvus==2.4.2

Note: you may need to restart the kernel to use updated packages.


In [2]:
from flask import Flask, request, render_template
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core import Settings
from dotenv import load_dotenv, find_dotenv
import os

In [3]:
_ = load_dotenv(find_dotenv())

In [4]:
documents = SimpleDirectoryReader("./data").load_data()
Settings.text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
vector_store = MilvusVectorStore(
    uri="./milvus_genco.db", dim=1536, overwrite=True
)

In [5]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, transformations=[SentenceSplitter(chunk_size=1024, chunk_overlap=20)]
)

In [6]:
query_engine = index.as_query_engine(similarity_top_k=2, model="gpt-4o-mini")

In [9]:
print( query_engine.query("What did President Nelson say about the Cardston Alberta Temple?"))

President Nelson mentioned that when he was born, there were six functioning temples in the Church, one of which was located in Cardston, Alberta, Canada.


In [7]:
app = Flask(__name__)

In [8]:
app = Flask(__name__)
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        query = request.form['query']
        response = query_engine.query(query)
        return render_template('index.html', query=query, response=response)
    return render_template('index.html')
if __name__ == '__main__':
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.18.167.202:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/bam/miniconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/bam/miniconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/bam/miniconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/bam/miniconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/bam/miniconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 331, in in

SystemExit: 1

/home/bam/miniconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
